<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/SimpleEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple embeddings

# Installation


In [ ]:
!pip install openai -q
!pip install supabase -q
!pip install pyodbc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 6.7 MB/s eta 0:00:00


## Get API Keys and setup

In [ ]:
from openai import OpenAI

from supabase import create_client, Client

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

import pandas as pd
import pyodbc

import requests
import json
from google.colab import userdata

# setup the gemini pro
# gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key=userdata.get('Gemini_API_Key'))

# setup Supabase connection
supabase_url = userdata.get('SUPABASE_URL')
supabase_key = userdata.get('SUPABASE_KEY')
supabase_client = create_client(supabase_url, supabase_key)

# OpenAI key
openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key=openai_api_key)


## Helper functions to run code


In [ ]:
#
# Retrieve a file from Github Repository

def get_ChinookDB_ER_from_github(file_url):

  # Send a GET request to the file URL
  response = requests.get(file_url)

  # Check if the request was successful
  if response.status_code == 200:
      # Get the content of the file
      file_content = response.text
      return file_content
  else:
      return(f'Failed to retrieve the file: {response.status_code}')

## Test connection to

- Supabase
- [https://github.com/lerocha/chinook-database](https://github.com/lerocha/chinook-database)

In [ ]:
## Testing connection to Supabase using Supabase client. A simple sanity check

response = supabase_client.table('Customer').select("*").execute()

## response object has data attribute that contains the result
## response.data
response_df = pd.DataFrame(response.data)
response_df.head(3)


## Get OpenAI assistant

In [ ]:
## OpenAI assistant and attached file details are shown at the end of this notebook

# Get OpenAI assistant
db_assistant = openai_client.beta.assistants.retrieve("asst_v3MqBfnVWdywefUUKPTIIB0P")

# Start a new thread to chat with OpenAI
thread = openai_client.beta.threads.create()

# Let's ask the first question - How many employees are there?
thread_message = openai_client.beta.threads.messages.create(
    thread_id = thread.id,
    role="user",
    content="How many employees are there?",
    )
run_1 = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=db_assistant.id)

In [ ]:
print(run_1)


Run(id='run_f6pMVmS6YX91jEvw9c0Zb5bz', assistant_id='asst_v3MqBfnVWdywefUUKPTIIB0P', cancelled_at=None, completed_at=None, created_at=1711848697, expires_at=1711849297, failed_at=None, file_ids=['file-nYMstH2zqNqOYe8N4YXqV7iI'], instructions='You are a Data Architect for ChinookDB. \n1. The mermaid ER script is in the attached file. \n2. Load and analyze the ER  script from the file provided by me.\n3. I will ask you to write SQL queries \n4. You will use this ER diagram to deliver SQL queries. \n5. Database is postgres. \n6. Use quotes for database object names in SQL\n7. Your response will be JSON object, containing two parts. SQL code and Explanation.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_rTx0GqPG1pL6v9N2439A4OE0', tools=[CodeInterpreterTool(type='code_interpreter')], usage=None, temperature=1.0)


In [ ]:
# Let's ask another question - How many employees are there?
thread_message = openai_client.beta.threads.messages.create(
    thread_id = thread.id,
    role="user",
    content="Give me total sales by genre for all genres",
    )
run_2 = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=db_assistant.id)

In [ ]:
print(run_2)

Run(id='run_XKEdOj22UFv1yxzb8hQuLiAc', assistant_id='asst_v3MqBfnVWdywefUUKPTIIB0P', cancelled_at=None, completed_at=None, created_at=1711848779, expires_at=1711849379, failed_at=None, file_ids=['file-nYMstH2zqNqOYe8N4YXqV7iI'], instructions='You are a Data Architect for ChinookDB. \n1. The mermaid ER script is in the attached file. \n2. Load and analyze the ER  script from the file provided by me.\n3. I will ask you to write SQL queries \n4. You will use this ER diagram to deliver SQL queries. \n5. Database is postgres. \n6. Use quotes for database object names in SQL\n7. Your response will be JSON object, containing two parts. SQL code and Explanation.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_rTx0GqPG1pL6v9N2439A4OE0', tools=[CodeInterpreterTool(type='code_interpreter')], usage=None, temperature=1.0)


In [ ]:
# Retrieve the chat history

history = openai_client.beta.threads.messages.list(thread_id=thread.id)

messages = history.data[::-1]
for i in messages:
    if(i.role.upper() == "USER"):
      print("\n","*"*60)
    print(i.role.upper() + ": "+ i.content[0].text.value)


 ************************************************************
USER: How many employees are there?
ASSISTANT: Based on the initial content of the ER diagram, it seems the diagram includes definitions for several entities like `Album`, `Artist`, and `Customer`, but doesn't directly display the `Employee` entity in the snippet provided. I will search for the `Employee` entity definition within the rest of the document to write an accurate SQL query.
ASSISTANT: The `Employee` entity is indeed part of the ER diagram. Now, I will proceed to write an SQL query to count the number of employees in the database.
ASSISTANT: ```json
{
  "SQL code": "SELECT COUNT(\"EmployeeId\") AS employee_count FROM \"Employee\";",
  "Explanation": "This SQL query is used to count the total number of employees in the 'Employee' table. It uses the COUNT aggregation function on the primary key 'EmployeeId' to ensure that only unique records are counted. The result is aliased as 'employee_count' for clarity."
}
```

In [ ]:
print(history.model_dump_json(indent=4))


{
    "data": [
        {
            "id": "msg_c0UzGc9w1weUPy2kKo5PslwK",
            "assistant_id": "asst_v3MqBfnVWdywefUUKPTIIB0P",
            "completed_at": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "```json\n{\n  \"SQL code\": \"SELECT g.\\\"Name\\\" AS genre, SUM(il.\\\"UnitPrice\\\" * il.\\\"Quantity\\\") AS total_sales FROM \\\"Genre\\\" g JOIN \\\"Track\\\" t ON g.\\\"GenreId\\\" = t.\\\"GenreId\\\" JOIN \\\"InvoiceLine\\\" il ON t.\\\"TrackId\\\" = il.\\\"TrackId\\\" GROUP BY g.\\\"GenreId\\\" ORDER BY total_sales DESC;\",\n  \"Explanation\": \"This SQL query calculates the total sales for each music genre. It does this by joining the 'Genre', 'Track', and 'InvoiceLine' tables using the appropriate foreign key relationships ('GenreId' and 'TrackId'). For each genre, it multiplies the 'UnitPrice' by the 'Quantity' of tracks sold (from the 'InvoiceLine' table) an

In [ ]:
# Inspect db_assistant
# print(db_assistant)
# print(db_assistant.data)
# print(db_assistant.model_dump_json(indent=4))

In [ ]:
# Display OpenAI Assistant info

print(db_assistant.model_dump_json(indent=4))

{
    "id": "asst_v3MqBfnVWdywefUUKPTIIB0P",
    "created_at": 1711830731,
    "description": null,
    "file_ids": [
        "file-nYMstH2zqNqOYe8N4YXqV7iI"
    ],
    "instructions": "You are a Data Architect for ChinookDB. The mermaid ER script is in the attached file. Load and analyze the ER  script from the file provided by me. I will ask you to write queries and you will use this ER diagram to deliver SQL queries. Database is postgres. use quotes for database object names in SQL",
    "metadata": {},
    "model": "gpt-4-turbo-preview",
    "name": "ChinookDB",
    "object": "assistant",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ]
}


In [ ]:
chinookDB_ER = get_ChinookDB_ER_from_github('https://raw.githubusercontent.com/cooolbabu/GoogleGemini101/main/OpenAI/ChinookDB_ER.txt')
print(chinookDB_ER)

erDiagram
    Album {
        integer AlbumId PK
        varchar Title
        integer ArtistId FK
    }

    Artist {
        integer ArtistId PK
        varchar Name
    }

    Customer {
        integer CustomerId PK
        varchar FirstName
        varchar LastName
        varchar Company
        varchar Address
        varchar City
        varchar State
        varchar Country
        varchar PostalCode
        varchar Phone
        varchar Fax
        varchar Email
        integer SupportRepId FK
    }

    Employee {
        integer EmployeeId PK
        varchar LastName
        varchar FirstName
        varchar Title
        integer ReportsTo FK
        datetime BirthDate
        datetime HireDate
        varchar Address
        varchar City
        varchar State
        varchar Country
        varchar PostalCode
        varchar Phone
        varchar Fax
        varchar Email
    }

    Genre {
        integer GenreId PK
        varchar Name
    }

    Invoice {
        intege